# Lesson 06: Integral transformation of river profiles

*This lesson has been written by Simon M. Mudd at the University of Edinburgh*

*Last update 30/09/2021*

In the last lesson, we looked at some of the drawbacks of using slope-area data to compare the steepness of channels. There is an alternative way to look at channel steepness that uses elevation rather than gradient of the channel. 

We start with Morisawa's law:

$S = k_s A^{-\theta}$

now remember that $S$ is the same as $dz/dx$, the derivative of elevation. $x$ is the flow distance. If we integrate this equation we get:
    
$z(x) = z(x_b) + \Big(\frac{k_s}{{A_0}^{\theta}}\Big) \int_{x_b}^{x} \Big(\frac{A_0}{A(x)}\Big)^{\theta} dx$

where $x_b$ is some arbitrary base level, and $A_0$ is a reference drainage area (this is to ensure the integrand is dimensionless). We almost always set $A_0$ to 1 $m^2$. The integrand seems annoying and messy, but it actually fairly easy to calculate from topographic data (you are just adding drainage area along the length of the channel. You will never need to do this yourself, there is software for calculating the integrand. It also has dimensions length. So we can define a coordinate, $\chi$:

$\chi = \int_{x_b}^{x} \Big(\frac{A_0}{A(x)}\Big)^{\theta} dx$

which is just that integrand, but it looks nicer in the equation:

$z(x) = z(x_b) + \Big(\frac{k_s}{{A_0}^{\theta}}\Big) \chi$

Now, have a look at that last equation. This is the equation of a line! And the gradient of the line, if $A_0$ = 1 $m^2$ is the channel steepness! 

## Wait, what does that mean?

It means that if you calculate $\chi$ (and we have software for that) you get a **transformed** profile where the gradient of the profile is the channel steepness. And actually, if you do a bunch of fancy mathematics, you can show that this works even if the erosion rates vary in time and space (see Royden and Perron, 2013).

The `channeltoy` includes the $\chi$ coordinate, so you can see what it looks like:

In [ ]:
!pip install channeltoy

In [ ]:
import channeltoy as ct
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Make a steady state channel
first_channel = ct.channeltoy(spacing=250, U = 0.0002, K = 0.00005, n=1, m= 0.45)
initial_elevation = first_channel.solve_steady_state_elevation()
initial_chi = first_channel.chi_data
x_locs  = first_channel.x_data

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(x_locs, initial_elevation,'b')
ax2.plot(initial_chi, initial_elevation,'r')


ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("elevation (m)")
ax1.title.set_text("Longitudinal profile")


ax2.set_xlabel("$\chi$ ($m$)")
ax2.set_ylabel("elevation (m)")
ax2.title.set_text("$\chi$ profile")

plt.tight_layout()


See how the chi ($\chi$) profile is straight? That is because this is a steady state channel. 
We can do a transient channel as well:

In [ ]:
# create a channel
chan = ct.channeltoy(spacing=20, U = 0.0002, K = 0.00005, n=1, m= 0.45)
initial_elevation = chan.solve_steady_state_elevation()

# change the uplift rate
chan.set_U_values(U = 0.0005)

# Run the transient simulation. You can use the start and end time to 
times, elevations = chan.transient_simulation(base_level = 0, dt = 50, 
                                              start_time = 0, end_time = 100001, 
                                              print_interval = 25000)

initial_chi = chan.chi_data
x_locs  = chan.x_data
z = elevations[-1]

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(x_locs, z,'b')
ax2.plot(initial_chi, z,'r')


ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("elevation (m)")


ax2.set_xlabel("$\chi$ ($m$)")
ax2.set_ylabel("elevation (m)")

plt.tight_layout()

See how the $\chi$ profile has two segments with a different gradient? The steeper part represents the section affected by the higher uplift rate in this transient run.

This example illustrates how we can look at $\chi$ profiles to see the relative steepness of channels.

## Some real $\chi$ profiles

Lets get that data from Xi'an, China that we have used in other lessons and look at it using `geopandas`

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
# Read in the data from Xi'an
df = pd.read_csv("Xian_chi_data_map.csv")
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf = gdf.set_crs(epsg=4326)
print(gdf.head())

Now plot the data

In [ ]:
# First lets isolate just one of these basins. They go from 0 to 12
gdf_b1 = gdf[(gdf['basin_key'] == 5)]

# Now make channel profile plots
z = gdf_b1.elevation
x_locs = gdf_b1.flow_distance
chi = gdf_b1.chi

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1, ax2) = plt.subplots(2, 1)
ax1.scatter(x_locs, z,s = 0.2)
ax2.scatter(chi, z,s = 0.2)


ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("elevation (m)")

ax2.set_xlabel("$\chi$ ($m$)")
ax2.set_ylabel("elevation (m)")

plt.tight_layout()

We can get rid of the tributaries as well:

In [ ]:
# First lets isolate just one of these basins. They go from 0 to 12
gdf_b1 = gdf[(gdf['basin_key'] == 3)]

# The main stem channel is the one with the minimum source key in this basin
min_source = np.amin(gdf_b1.source_key)
gdf_b2 = gdf_b1[(gdf_b1['source_key'] == min_source)]

# Now make channel profile plots
z = gdf_b2.elevation
x_locs = gdf_b2.flow_distance
chi = gdf_b2.chi

# Create two subplots and unpack the output array immediately
plt.clf()
f, (ax1, ax2) = plt.subplots(2, 1)
ax1.scatter(x_locs, z,s = 0.2)
ax2.scatter(chi, z,s = 0.2)


ax1.set_xlabel("Distance from outlet ($m$)")
ax1.set_ylabel("elevation (m)")

ax2.set_xlabel("$\chi$ ($m$)")
ax2.set_ylabel("elevation (m)")

plt.tight_layout()

__Task:__ In the prevous two code snippets, vary the basin number and look for knickpoints.